# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [24]:
import openai
import os
from dotenv import load_dotenv

def set_env_path(env_path):
    # Set the path to the .env file
    load_dotenv(dotenv_path=env_path)

# Define the path to the .env file
env_path = "YOUR_ENV_FILE_PATH"
# Set the environment variables from the .env file
set_env_path(env_path)

# Access the OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')




In [3]:
!pip install pandas

In [47]:
import pandas as pd
df = pd.read_csv('/Users/m.maguga-darbinian/Data.csv')

In [48]:
df.head()

,Input number,Product,Review
0,0,Smartwatch,"""I purchased this smartwatch recently, and I'm..."
1,1,Bluetooth Speaker,"""The sound quality of this Bluetooth speaker i..."
2,2,Laptop,"""This laptop is lightning fast and perfect for..."
3,3,Headphones,"""These headphones provide excellent audio qual..."
4,4,Camera,"""I love the image clarity and the variety of s..."


## LLMChain

In [49]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [50]:
llm = ChatOpenAI(temperature=0.9)

In [51]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

Now, we will combine into a chain

In [52]:
chain = LLMChain(llm=llm, prompt=prompt)

In [53]:
product = "A product Management company helping other companies implement AI capabilities within their products"
chain.run(product)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


'AI Innovations'

## SimpleSequentialChain
Run a sequence of chains one after the other. Single input & single output

In [54]:
from langchain.chains import SimpleSequentialChain

In [55]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [56]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [57]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [58]:
overall_simple_chain.run(product)



> Entering new  chain...
AI Enablement Solutions
AI Enablement Solutions is a leading provider of cutting-edge technology solutions that empower businesses to harness the power of artificial intelligence.

> Finished chain.


'AI Enablement Solutions is a leading provider of cutting-edge technology solutions that empower businesses to harness the power of artificial intelligence.'

## SequentialChain
When multiple inputs and outputs

In [59]:
from langchain.chains import SequentialChain

In [60]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [61]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [62]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [63]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [64]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [68]:
review = df.Review[0]
overall_chain(review)



> Entering new  chain...

> Finished chain.


{'Review': '"I purchased this smartwatch recently, and I\'m impressed with its sleek design and features."',
 'English_Review': '"I recently bought this smartwatch, and I\'m amazed by its stylish design and functionalities."',
 'summary': "The reviewer is extremely impressed by the smartwatch's sleek design and impressive range of features.",
 'followup_message': 'Response: Thank you for your positive review of our smartwatch! We are thrilled to hear that you are extremely impressed with its sleek design and extensive range of features. We strive to offer our customers the best possible experience, and your feedback reassures us that we are on the right track. If you have any further comments or suggestions, please feel free to share. We appreciate your support and hope you continue to enjoy using our smartwatch.'}

In [71]:
review = df.Review[2 & 3]
overall_chain(review)



> Entering new  chain...

> Finished chain.


{'Review': '"This laptop is lightning fast and perfect for my work. The battery life is exceptional."',
 'English_Review': 'Este portátil es increíblemente rápido y perfecto para mi trabajo. La duración de la batería es excepcional.',
 'summary': 'This laptop is incredibly fast and perfect for my work, with exceptional battery life.',
 'followup_message': "Response: Thank you for your positive review! We're glad to hear that the laptop's speed has been impressive and that it is perfectly suited for your work needs. Additionally, we're delighted to know that the battery life has exceeded your expectations. We strive to provide efficient and reliable products, and your feedback is greatly appreciated. If you have any further questions or concerns, please feel free to reach out."}

## Router Chain
To rout the input to a chain, depending the topic 


In [72]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [73]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [74]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [75]:
llm = ChatOpenAI(temperature=0.0)

In [76]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [77]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

Instructions of the task to be done, and formatting of the output

In [78]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

RouterOutputParser = It will help the chain decide which subchains to rout between.

In [79]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [80]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [82]:
chain.run("What is black body radiation?")



> Entering new  chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation refers to the electromagnetic radiation emitted by an object that absorbs all incident radiation and reflects or transmits none. It is called "black body" because it absorbs all wavelengths of light, appearing black at room temperature. \n\nAccording to Planck\'s law, black body radiation is characterized by a continuous spectrum of wavelengths and intensities, which depend on the temperature of the object. As the temperature increases, the peak intensity of the radiation shifts to shorter wavelengths, resulting in a change in color from red to orange, yellow, white, and eventually blue at very high temperatures.\n\nBlack body radiation is a fundamental concept in physics and has significant applications in various fields, including astrophysics, thermodynamics, and quantum mechanics. It played a crucial role in the development of quantum theory, as Max Planck\'s explanation of black body radiation led to the birth of quantum mechanics.'

In [83]:
chain.run("What is 2+1")



> Entering new  chain...
math: {'input': 'What is 2+1'}
> Finished chain.


"Thank you for your kind words! I'm here to help with any math questions you have.\n\nTo answer your question, 2 + 1 equals 3."

In [84]:
chain.run("How do I learn about python?")



> Entering new  chain...
computer science: {'input': 'How do I learn about python?'}
> Finished chain.


'Learning about Python can be an exciting journey! Here are some steps you can follow to get started:\n\n1. Familiarize yourself with the basics: Begin by understanding the fundamental concepts of Python, such as variables, data types, control structures (if-else, loops), functions, and modules. Online tutorials, interactive coding platforms, or Python documentation can be helpful resources.\n\n2. Practice coding: The best way to learn Python is by writing code. Start with simple exercises and gradually move on to more complex problems. Websites like HackerRank, LeetCode, or Codecademy offer coding challenges and exercises specifically designed for Python learners.\n\n3. Read Python documentation: Python has extensive documentation available online, which provides detailed explanations of its features, libraries, and modules. The official Python documentation (docs.python.org) is a valuable resource to understand the language\'s syntax, standard library, and best practices.\n\n4. Explo

In [85]:
chain.run("What is the capital of the Netherlands?")



> Entering new  chain...
None: {'input': 'What is the capital of the Netherlands?'}
> Finished chain.


'The capital of the Netherlands is Amsterdam.'